# Implicit hydrogen bond interaction
This tutorial showcases how to calculate hydrogen bond (H-bond) interations for a topology with only heavy atoms (implicit hydrogens). Also, the difference between the implicit and explicit methods are demonstrated.
 
Before getting started to the implicit H-bond calculation, we need to make sure the bond orders of molecules are corrected. We introduce a protein helper function (`ProteinHelper`) to do all in once. 

## Protein helper function
The protein helper function is designed to fix the bond orders in your topology file based on a given molecule template. The algorithms of fixing the bond orders are inspired by [pdbinf](https://github.com/OpenFreeEnergy/pdbinf) and RDKit's function (`rdkit.Chem.AllChem.AssignBondOrdersFromTemplate`). 


### A small example: ACE-TPO-NME
This example is originally come from [pdbinf](https://github.com/OpenFreeEnergy/pdbinf). This example shows a molecule with three residues: [ACE](https://www.rcsb.org/ligand/ACE), [TPO](https://www.rcsb.org/ligand/TPO), [NME](https://www.rcsb.org/ligand/NME). You can find the correct bond orders at [RCSB PDB](https://www.rcsb.org/).

In [ ]:
from rdkit import Chem
from rdkit.Chem.rdDetermineBonds import DetermineConnectivity

from prolif import Molecule
from prolif.datafiles import datapath

protein_path = datapath / "protein_helper/tpo.pdb"

# Read the data with RDKit
mol = Chem.MolFromPDBFile(str(protein_path), removeHs=False)
DetermineConnectivity(mol, useHueckel=True)
for atm in mol.GetAtoms():
    atm.SetNoImplicit(False)  # set no implicit to False

# Convert to ProLIF Molecule
mol = Molecule.from_rdkit(mol)
print([res for res in mol.residues])
mol

In [ ]:
import prolif as plf

plf.display_residues(mol)

Considering these three residues are not standard protein residues (20 amino acids), you will need to provide a template for the additional residues. We provide two ways to load your templates into your protein helper function.
1. CIF template: you can look up your residue (ligand) at [RCSB PDB](https://www.rcsb.org/)'s ligand page and download it by clicking *Download Files > Definition (CIF format)*.
2. SMILES string template: simply provide a SMILES string for your residue or ligand.

In [ ]:
from prolif.datafiles import datapath
from prolif.io.cif import cif_template_reader
from prolif.io.protein_helper import ProteinHelper

# load templates
ace_template = {"ACE": {"SMILES": "CC=O"}}
tpo_template = {"TPO": {"SMILES": "C[C@H]([C@@H](C(=O))N)OP(=O)(O)O"}}
# tpo_template = cif_template_reader(
#     datapath / "protein_helper/templates/TPO.cif"
# )  # alternative
nme_template = cif_template_reader(datapath / "protein_helper/templates/NME.cif")

# fix molecule by the templates
protein_helper = ProteinHelper(templates=[ace_template, tpo_template, nme_template])

````{important}
If you would like to use SMILES templates, you have to first indicate your residue name, and then define your SMILES string in a dictionary with key `SMILES`. For instance:
```python
{
    "[residue name]": {
        "SMILES": "[your smiles string]"
    },
    ...
}
```
````

```{tip}
If using SMILES templates, you might need to **modify your SMILES string by pruning one oxygen in an acetate group** (especially, if your residue is not in the terminus). For instance in the above example, original `TPO` residue's SMILES is *C\[C@H\](\[C@@H\](C(=O)**O**)N)OP(=O)(O)O* (if you look at [RCSB TPO page](https://www.rcsb.org/ligand/TPO)), but your template should be *C\[C@H\](\[C@@H\](C(=O))N)OP(=O)(O)O*. It is because prolif splits your molecule into residues at the amide bonds before calculating molecular interactions. The `ProteinHelper` is correcting your **residues**, so you actually have to provide templates without `-OH` in the acetate group forming amide bond.
```


Let's fix the residues.

In [ ]:
# fix molecule by the templates
fixed_mol = protein_helper.standardize_protein(mol)

plf.display_residues(mol)

Now, it works.

After learning how to use protein helper function, we can now learn to calculate the implicit H-bond interactions.

## Implicit H-bond interactions
First, we read the receptor (protein) and fix the bond orders with `ProteinHelper`.

In [ ]:
# read the receptor
from rdkit import Chem

from prolif.datafiles import datapath
from prolif.io.protein_helper import ProteinHelper

data_path = datapath / "implicitHbond"
protein_path_i = data_path / "receptor.pdb"
protein_mol_i = Molecule.from_rdkit(Chem.MolFromPDBFile((str(protein_path_i))))

protein_helper = ProteinHelper()

# fix the receptor
protein_mol_i = protein_helper.standardize_protein(protein_mol_i)

plf.display_residues(protein_mol_i, slice(100, 108))

Don't worry if there is a lot of warning message generated. It is because your topology file is not complete for some residues (e.g., only have backbone and miss some heavy atoms at the side chain). If the residues are not close to your ligand, it should be fine.

```{note}
The protein helper function will guess the forcefield by residue name. Because for some forcefield a certain residue name can represent a specific type of protonated state of the residue. The protein function will fix your residue to that corresponding protonated states.

In the above example, the protein helper function fails to guess the forcefield, and thus it assigns the residues' protonated states based on default settings (basically, pH = 7 and CYS is neutralized).
```

```{tip}
You can modify the residue name for your residue to get different protonated state of the residue when using protein helper function. This approach is particularly useful when investigating different protonation states of residues.
```

Next, we read the ligand. Because the file is in `.sdf` format, we used `prolif.sdf_supplier` to read it.

In [ ]:
# Read the ligand
ligand_i = plf.sdf_supplier(data_path / "1.D.sdf")[0]
plf.display_residues(ligand_i, size=(400, 200))

We then used `prolif.Fingerprint` to find the implicit hydrogen bonds by setting `interactions= ["ImplicitHBAcceptor", "ImplicitHBDonor"]`. Some parameters for the implicit H-bond method can be set. For instance, if you want to include water for detecting interactions, use `"include_water": True`. 

Geometric checks play an important role in the implicit hydrogen bond method by filtering out impossible interaction pairs. You can make the method more flexible by adjusting the tolerance thresholds for acceptor or donor atom angles and plane angles (plane angles only for sp2-hybridized heavy atoms). This allows you to include more or fewer interactions based on your needs. Increasing the tolerance reduces the geometric restrictions. Our default settings are tuned by a dataset from RCSB PDB. If you don't want any geometry check, you can simply apply `"ignore_geometry_checks": True`.  

Then, we used `run_from_iterable` to calculate the implicit H-bond interactions.

In [ ]:
fp_i = plf.Fingerprint(
    interactions=["ImplicitHBAcceptor", "ImplicitHBDonor"],
    count=True,
    parameters={
        "ImplicitHBAcceptor": {
            "include_water": False,  # include water residues in the detection
            "tolerance_dev_aaa": 30,  # atom angle deviation tolerance for acceptor
            "tolerance_dev_daa": 30,  # atom angle deviation tolerance for donor
            "tolerance_dev_dpa": 30,  # plane angle deviation tolerance for donor
            "tolerance_dev_apa": 90,  # plane angle deviation tolerance for acceptor
            "ignore_geometry_checks": False,  # skip geometry checks
        },
        "ImplicitHBDonor": {
            "include_water": False,  # include water residues in the detection
            "tolerance_dev_aaa": 30,  # angle deviation tolerance for acceptor atom
            "tolerance_dev_daa": 30,  # angle deviation tolerance for donor atom
            "tolerance_dev_dpa": 30,  # angle deviation tolerance for donor plane
            "tolerance_dev_apa": 90,  # angle deviation tolerance for acceptor plane
            "ignore_geometry_checks": False,  # skip geometry checks
        },
    },
)
fp_i.run_from_iterable([ligand_i], protein_mol_i)

Similar to the previous tutorials, we converted the fingerprint into a table.

In [ ]:
df_i = fp_i.to_dataframe().T
df_i

### Metadata

If you would like to have more details (metadata), use `.ifp` with the key `("[ligand's resname][resid].[chainid]", "[protine's resname][resid].[chainid]")`. Mostly, metadata are related to the geometric checks, including raw atom and plane angles and their deviations. We also included `vina_hbond_potential` to assess the likelihood of an interaction occurring. Higher values of `vina_hbond_potential` suggest a stronger probability of the interaction.

In [ ]:
fp_i.ifp[0][("UNL1", "TYR167.B")]

````{tip}
If you have underscore (`_`) in your chain ID, you might have `KeyError`. Try to use `ResidueId` rather than `string` format as the dictionary key.

```python
>>> fp_i.ifp[0][("UNL1", "HOH1._")] # Not working
KeyError: (ResidueId(UNL, 1, None), ResidueId(HOH, 1, None))

# try this
>>> from prolif.residue import ResidueId

>>> fp_i.ifp[0][(ResidueId("UNL", 1, None), ResidueId("HOH", 1, "_"))]
{'ImplicitHBAcceptor': ... }
```
````

### Visualization

For 2D ligand-centered plot, use `plot_lignetwork` with your ligand.

In [ ]:
view = fp_i.plot_lignetwork(ligand_i, kind="frame", frame=0, display_all=True)
view

For a protein-ligand complex, `plot_3d` is a better way to explore the interactions. It is noted that you might need to set the water molecules into `"sphere"` to visualize it.

In [ ]:
view = fp_i.plot_3d(
    ligand_i, protein_mol_i, frame=0, display_all=True, remove_hydrogens=False
)
view.setStyle(
    {
        "resn": "HOH",
    },
    {"sphere": {"radius": 0.5, "color": "red"}},
)
view

## Explicit H-bond interaction method (standard method)
As a reference, we also calculated the explicit H-bond interactions for the same system.

In [ ]:
import MDAnalysis as mda

from prolif import Molecule
from prolif.datafiles import datapath

# Read the protonated protein file
data_path = datapath / "implicitHbond"
protein_path = data_path / "receptor_ph7_amber2.pdb"
u = mda.Universe(str(protein_path))
protein_mol = Molecule.from_mda(u)

plf.display_residues(protein_mol, slice(100, 108))

In [ ]:
# Read the protonated ligand file
ligand = plf.sdf_supplier(str(data_path / "1.D_protonated.sdf"))[0]
plf.display_residues(ligand, size=(400, 200))

In [ ]:
# Calculate default H-bond interactions
fp_count = plf.Fingerprint(["HBDonor", "HBAcceptor"], count=True)
fp_count.run_from_iterable([ligand], protein_mol)

In [ ]:
# Get the fingerprint DataFrame
df = fp_count.to_dataframe().T
df

### 2D/3D Visualization

In [ ]:
view = fp_count.plot_lignetwork(ligand, kind="frame", frame=0, display_all=True)
view

In [ ]:
view = fp_count.plot_3d(
    ligand, protein_mol, frame=0, display_all=False, remove_hydrogens=False
)
view

# Comparison between explicit and implicit H-bond interaction methods
Finally, let’s compare the explicit and implicit hydrogen bond interaction methods. As shown in the visualization below, the implicit method tends to detect more interactions. This is because the explicit hydrogen bond method relies on how hydrogens are added to the ligand and protein structures. For some rotatable hydrogen donors, you can imagine these donors as being flexible during molecular dynamics simulations. The explicit method typically captures a single conformational state of hydrogen bond interactions, whereas the implicit method can be seen as integrating results from multiple conformational states. This is the fundamental difference between the two methods, and neither is inherently superior. The choice depends on the type of information you aim to extract from your data.

In [ ]:
from prolif.plotting.complex3d import Complex3D

# create Complex3D objects (default)
comp3D = Complex3D.from_fingerprint(fp_count, ligand, protein_mol, frame=0)
# (modified)
other_comp3D = Complex3D.from_fingerprint(fp_i, ligand_i, protein_mol_i, frame=0)

# compare the two Complex3D objects
view = comp3D.compare(other_comp3D, display_all=True)
view